In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import nltk

import string
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd

# Provide the path to your Excel file
excel_file_path = '/content/drive/MyDrive/Financial Statement Analysis /Financial Statement Analysis Code/finphrase_fica_dataset.xlsx'

# Read the Excel file and convert it to a DataFrame
df = pd.read_excel(excel_file_path)

# Display the DataFrame
print(df)

                                               Sentence Sentiment
0     The GeoSolutions technology will leverage Bene...  positive
1     $ESI on lows, down $1.50 to $2.50 BK a real po...  negative
2     For the last quarter of 2010 , Componenta 's n...  positive
3     According to the Finnish-Russian Chamber of Co...   neutral
4     The Swedish buyout firm has sold its remaining...   neutral
...                                                 ...       ...
5837  RISING costs have forced packaging producer Hu...  negative
5838  Nordic Walking was first used as a summer trai...   neutral
5839  According shipping company Viking Line , the E...   neutral
5840  In the building and home improvement trade , s...   neutral
5841  HELSINKI AFX - KCI Konecranes said it has won ...  positive

[5842 rows x 2 columns]


In [ ]:
def cleanText(text):
    lemma = WordNetLemmatizer()
    stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having',  'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',  'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', ]

    text = text.lower()
    words = nltk.word_tokenize(text)
    words = ' '.join([lemma.lemmatize(word) for word in words if word not in stopwords])
    text = "".join(words)
  # removing non-alphabetic characters
    txt = re.sub('[^a-z]',' ',text)
    return txt

In [ ]:
 df['Cleaned_text'] = df['Sentence'].apply(cleanText) # Considered all the sample
 df.head()

,Sentence,Sentiment,Cleaned_text
0,The GeoSolutions technology will leverage Bene...,positive,geosolutions technology leverage benefon s gp...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi low bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,last quarter componenta s net sale dou...
3,According to the Finnish-Russian Chamber of Co...,neutral,according finnish russian chamber commerce m...
4,The Swedish buyout firm has sold its remaining...,neutral,swedish buyout firm sold remaining percen...


In [ ]:
X = df.Cleaned_text
y = df.Sentiment

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20,stratify=y, random_state=0)

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

In [ ]:
#Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
#  Design the neural network architecture
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_tfidf.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
#  Train the neural network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                532736    
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 3)                 99        
                                                                 
Total params: 534,915
Trainable params: 534,915
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:


model.fit(X_train_tfidf, y_train_encoded, epochs=10, batch_size=32)

Epoch 1/10
147/147 [==============================] - 2s 7ms/step - loss: 0.9447 - accuracy: 0.5622
Epoch 2/10
147/147 [==============================] - 1s 6ms/step - loss: 0.6067 - accuracy: 0.7417
Epoch 3/10
147/147 [==============================] - 1s 9ms/step - loss: 0.3313 - accuracy: 0.8536
Epoch 4/10
147/147 [==============================] - 1s 10ms/step - loss: 0.2289 - accuracy: 0.8817
Epoch 5/10
147/147 [==============================] - 1s 7ms/step - loss: 0.1829 - accuracy: 0.8934
Epoch 6/10
147/147 [==============================] - 1s 7ms/step - loss: 0.1632 - accuracy: 0.9007
Epoch 7/10
147/147 [==============================] - 1s 7ms/step - loss: 0.1519 - accuracy: 0.9033
Epoch 8/10
147/147 [==============================] - 1s 6ms/step - loss: 0.1476 - accuracy: 0.9001
Epoch 9/10
147/147 [==============================] - 1s 8ms/step - loss: 0.1415 - accuracy: 0.9007
Epoch 10/10
147/147 [==============================] - 2s 11ms/step - loss: 0.1356 - accuracy: 0.90

In [ ]:
y_pred = model.predict(X_test_tfidf)
y_pred = np.argmax(y_pred, axis=1)
y_pred = label_encoder.inverse_transform(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

37/37 [==============================] - 0s 5ms/step
Accuracy: 0.6458511548331908
